In [1]:
import six
import numpy as np

In [269]:
DATA = """
5 + 6 * 9 + 5 + 5 + (7 * 2)
2 + (3 + 7 + 7) * 5 + 9 + (5 + (4 + 5 + 3 + 6) + 8 * 5 + (7 + 4 + 7 + 6 + 9) + 9)
(5 * 4 + 7 + 4 + 8) + 9 * ((5 * 4) + 2 + (6 + 2 + 8 + 7 + 4 + 5)) * 5
4 * ((4 + 9 + 3 * 3 * 5) * 3 + 2 + 2 * (9 * 2 + 3)) + 5 * ((8 * 3 * 6) + 9 * 4 + (5 + 4 * 7 + 7)) * 7 * (9 + 2 * 2)
5 * 6 + ((8 + 9) * 5 * 9 + 4 + (9 + 3 * 9))
(3 * (3 + 3) * 9 + (6 + 9 * 8) + 5) + 5
6 * (9 * 6 * 9)
""".strip()

In [270]:
def solve(tokens):
    
    """
    Given the token representation of the expression, it
    evaluates it to calculate the result.
    
    :param tokens: list(str|int|list)
    :return: int
    """
    
    def evaluate(x):
        if isinstance(x, int): return x
        elif isinstance(x, list): return solve(x)
        else: raise
    
    i, value = 1, evaluate(tokens[0])
    
    while i < len(tokens):
        token = tokens[i]
        if   token == '+': value += evaluate(tokens[i+1])
        elif token == '*': value *= evaluate(tokens[i+1])
        elif token == '/': value /= evaluate(tokens[i+1])
        elif token == '-': value -= evaluate(tokens[i+1])
        else: raise
        i += 2
        
    return value

In [271]:
def priority(tokens, operators=None):
    
    """
    Given the token representation of the expression, it
    applies the prioritisation rules for the given (or 
    default operators).
    
    :param tokens: list(str|int|list)
    :return: list(str|int|list)
    """
    
    operators = operators or ('+',)
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if isinstance(token, list):
            tokens[i] = priority(token)
            i += 1
        elif token in operators:
            token = tokens[i-1:i+1] + [priority(tokens[i+1]) if isinstance(tokens[i+1], list) else tokens[i+1]]
            tokens = tokens[:i-1] + tokens[i+2:]
            tokens.insert(i-1, token)
            i += 0
        else:
            i += 1
    return tokens

In [272]:
def tokenise(expression):
    
    """
    Given a numerical expreession comprised of numbers,
    operators and parenthesis, it constructs a tokenised
    tree representation.
    
    :param expression: str
    :return: list(str|int|list)
    """
    
    expression = expression.replace('(', '( ').replace(')', ' )')
    tokens = [int(token) if token.isdigit() else token for token in expression.split(' ')]   
    fetch = lambda lst, depth: fetch(lst[-1], depth-1) if depth else lst
    atokens, level = [], 0
    for token in tokens:
        if token == '(':
            fetch(atokens, level).append([])
            level += 1
        elif token == ')': level -= 1
        else: fetch(atokens, level).append(token)                 
    return atokens

In [273]:
calculate = lambda expression: solve(tokenise(expression))

In [274]:
sum(calculate(e) for e in DATA.splitlines())

6387925487

In [275]:
calculate = lambda expression: solve(priority(tokenise(expression)))

In [276]:
sum(calculate(e) for e in DATA.splitlines())

926330723780